In [1]:
import json
from web3 import Web3
from bip44 import Wallet
from bip44.utils import get_eth_addr

In [2]:
MNEMONIC = "test test test test test test test test test test test junk"
w = Wallet(MNEMONIC)
sk, pk = w.derive_account("eth", address_index=0)
sender = get_eth_addr(pk)
ZERO_ADDRESS = "0x0000000000000000000000000000000000000000"
ETHER = int(1e18)

# Minimal exec fee that is executed automatically by debridge oracles
DEBRIDGE_EXEC_FEE = int(0.02 * ETHER)

In [3]:
kovan_provider = Web3(Web3.HTTPProvider("https://kovan.poa.network"))

In [6]:
print(f"Your address: {sender}")
eth_balance = kovan_provider.eth.get_balance(sender)
print(f"Your balance is {eth_balance/1e18} ETH")
if eth_balance < 0.5 * ETHER:
    print(f"You need to top up your account first via faucet")
    print(f"https://faucet.paradigm.xyz/")

Your address: 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
Your balance is 0.9463974849948396 ETH


In [7]:
with open('abis/DeBridgeGate.json') as json_file:
    gate_abi = json.load(json_file)

with open('abis/L1Bridge.json') as json_file:
    bridge_abi = json.load(json_file)

with open('abis/L1Token.json') as json_file:
    token_abi = json.load(json_file)

In [8]:
gate = kovan_provider.eth.contract(address='0x68D936Cb4723BdD38C488FD50514803f96789d2D', abi=gate_abi['abi'])

In [9]:
global_fixed_nativeFee = gate.functions.globalFixedNativeFee().call()
global_fixed_nativeFee

10000000000000000

In [11]:
bridge = kovan_provider.eth.contract(address='0x279e0cF58d6B3913C917222A61ba9ef5E9fF8057', abi=bridge_abi['abi'])

In [12]:
l1_token_address = bridge.functions.l1Token().call()
l1_token_address

'0x55904F2C4B672103a4E04E6BFcb1c5DDC4f41D6F'

In [14]:
l1_token = kovan_provider.eth.contract(address=l1_token_address, abi=token_abi['abi'])
print(l1_token.address)

0x55904F2C4B672103a4E04E6BFcb1c5DDC4f41D6F


In [17]:
# mint token if account doesn't have one

from web3.exceptions import (ContractLogicError,)

# find first token owned by the sender (with index0)
try:
    token_id = l1_token.functions.tokenOfOwnerByIndex(sender, 0).call()
    token_id
    print(f"You have at least one token on your balance. Id: {token_id}")
    need_mint = False
except:
    print("it seems you have no tokens yet. We should mint one...")
    need_mint = True

if need_mint:
    token_id = 0
    # We should find non-minted token_id first
    while True:
        try:
            l1_token.functions.ownerOf(token_id).call()
            token_id += 1
        except ContractLogicError:
            print(token_id, "doesn't exist")
            break

    print("minting it...")
    tx = l1_token.functions.mint(sender, token_id).buildTransaction({"from": sender, "nonce": kovan_provider.eth.get_transaction_count(sender)})
    raw_tx = kovan_provider.eth.account.sign_transaction(tx, sk).rawTransaction
    tx_hash = kovan_provider.eth.send_raw_transaction(raw_tx)
    kovan_provider.eth.wait_for_transaction_receipt(tx_hash)
    print(tx_hash.hex(), " mined so token should be minted")

You have at least one token on your balance. Id: 23


In [18]:
# Check if tokens are approved for bridge gateway

if (not l1_token.functions.isApprovedForAll(sender, bridge.address).call()):
    print("not approved yet. Approving")
    tx = l1_token.functions.setApprovalForAll(bridge.address, True).buildTransaction({"from": sender, "nonce": kovan_provider.eth.get_transaction_count(sender)})
    raw_tx = kovan_provider.eth.account.sign_transaction(tx, sk).rawTransaction
    tx_hash = kovan_provider.eth.send_raw_transaction(raw_tx)
    kovan_provider.eth.wait_for_transaction_receipt(tx_hash)
    print(tx_hash.hex(), "mined")
else:
    print("already approved")

already approved


In [19]:
value = global_fixed_nativeFee + DEBRIDGE_EXEC_FEE
value

30000000000000000

In [20]:
tx = bridge.functions.outboundTransfer(sender, token_id).buildTransaction({"from": sender, "nonce": kovan_provider.eth.get_transaction_count(sender),
 "gasPrice": 10_000_000_000, "gas": 3_000_000, "value": 50000000000000000 })
tx

{'chainId': 42,
 'from': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
 'nonce': 2637,
 'gasPrice': 10000000000,
 'gas': 3000000,
 'value': 50000000000000000,
 'to': '0x279e0cF58d6B3913C917222A61ba9ef5E9fF8057',
 'data': '0x8ba9c828000000000000000000000000f39fd6e51aad88f6f4ce6ab8827279cfffb922660000000000000000000000000000000000000000000000000000000000000017'}

In [21]:
raw_tx = kovan_provider.eth.account.sign_transaction(tx, sk).rawTransaction

In [22]:
tx_hash = kovan_provider.eth.send_raw_transaction(raw_tx)
tx_hash

HexBytes('0xdaa29ad497ca52dab49ef30bea05bee1b81e9f7792fa923c5316ce349fe521ad')

In [24]:
send_receipt = kovan_provider.eth.wait_for_transaction_receipt(tx_hash)
if len(send_receipt.logs) < 5 and send_receipt.status == 1:
    print(f"Tx {tx_hash.hex()} failed.")
    print(send_receipt)
else:
    print(f"Tx {tx_hash.hex()} Succeeded.")
    print("See it on DeBridge dashboard: https://testnet.debridge.finance/transaction?tx=" + tx_hash.hex() + "&chainId=42")

Tx 0xdaa29ad497ca52dab49ef30bea05bee1b81e9f7792fa923c5316ce349fe521ad Succeeded.
See it on DeBridge dashboard: https://testnet.debridge.finance/transaction?tx=0xdaa29ad497ca52dab49ef30bea05bee1b81e9f7792fa923c5316ce349fe521ad&chainId=42
